In [1]:
import anndata
import torch
import stPlus
import os
import random
import warnings
import pickle

import squidpy as sq
import numpy as np
import scanpy as sc
import pandas as pd
import spatialdm as sdm

from sklearn.model_selection import KFold
from matplotlib import pyplot as plt
from transpa.eval_util import calc_corr
from transpa.util import expTransImp, leiden_cluster, compute_autocorr, plot_genes
# from benchmark import SpaGE_impute, Tangram_impute
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score, normalized_mutual_info_score, homogeneity_score
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
from exp_spatialdm import spatialdm


warnings.filterwarnings('ignore')
pre_datapaths = ["../../output/preprocessed_dataset/seqFISH_single_cell.pkl",
                 "../../output/preprocessed_dataset/merfish_moffit.pkl",
                 "../../output/preprocessed_dataset/osmFISH_allenvisp.pkl",
                 "../../output/preprocessed_dataset/starmap_allenvisp.pkl"
                 ]

seed = 10
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

/illumina/scratch/deep_learning/cqiao/softwares/anaconda/envs/py311/lib/python3.11/site-packages/torch/__init__.py:690: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at /opt/conda/conda-bld/pytorch_1704987615470/work/torch/csrc/tensor/python_tensor.cpp:451.)
  _C._set_default_tensor_type(t)


In [2]:
pre_datapath = pre_datapaths[0]
with open(pre_datapath, 'rb') as infile:
    spa_adata, scrna_adata, raw_spatial_df, raw_scrna_df, raw_shared_gene = pickle.load(infile)
    cls_key = 'leiden'
    classes = scrna_adata.obs[cls_key]
    ct_list = np.unique(classes)

In [3]:
sc.pp.highly_variable_genes(scrna_adata, n_top_genes=3000)
scrna_adata

AnnData object with n_obs × n_vars = 32844 × 29452
    obs: 'cell', 'barcode', 'sample', 'pool', 'stage', 'sequencing.batch', 'theiler', 'doub.density', 'doublet', 'cluster', 'cluster.sub', 'cluster.stage', 'cluster.theiler', 'stripped', 'celltype', 'colour', 'sizeFactor', 'leiden'
    var: 'ENSEMBL', 'SYMBOL', 'SymbolUniq', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'log1p', 'hvg'

In [4]:
# Filter small/tiny celltypes (# cells <= 200)
# Find celltype specific genes
tp, counts = np.unique(scrna_adata.obs.celltype, return_counts=True)
sub_scrna_adata = scrna_adata[scrna_adata.obs.celltype.isin([_tp for _tp, _ct in zip(tp, counts) if _ct > 200]),scrna_adata.var['highly_variable']]
sc.tl.rank_genes_groups(sub_scrna_adata, 'celltype', method='wilcoxon')
sc.tl.filter_rank_genes_groups(sub_scrna_adata, min_fold_change=2)

In [5]:
# Fetch the top 50 scored celltype marker genes
top_k = 50
candidate_genes = set()
for i in range(top_k):
    for g in sub_scrna_adata.uns['rank_genes_groups_filtered']['names'][i]:
        if type(g) != str: continue
        candidate_genes.add(g)
print(f"# unique sc celltype marker genes: {len(candidate_genes)}")

# unique sc celltype marker genes: 422


In [6]:
extra_genes = np.setdiff1d(list(candidate_genes), raw_shared_gene)
print(f'# extra genes not in Spa ST: {len(extra_genes)}')

# extra genes not in Spa ST: 267


In [7]:
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity
res = expTransImp(
        df_ref=raw_scrna_df,
        df_tgt=raw_spatial_df,
        train_gene=raw_shared_gene,
        test_gene=np.concatenate([raw_shared_gene, extra_genes]),
        n_simulation=200,
        signature_mode='cell',
        mapping_mode='lowrank',
        classes=classes,
        n_epochs=2000,
        seed=seed,
        device=device)

[TransImp] Epoch: 2000/2000, loss: 0.781017, (IMP) 0.781017: 100%|██████████| 2000/2000 [00:30<00:00, 65.33it/s]


In [8]:
spa_adata

AnnData object with n_obs × n_vars = 57536 × 351
    obs: 'uniqueID', 'embryo', 'pos', 'z', 'x_global', 'y_global', 'x_global_affine', 'y_global_affine', 'embryo_pos', 'embryo_pos_z', 'Area', 'UMAP1', 'UMAP2', 'celltype_mapped_refined', 'segmentation_vertices_x_global_affine', 'segmentation_vertices_y_global_affine'
    uns: 'log1p', 'spatial_neighbors'
    obsm: 'spatial'
    layers: 'normalized'
    obsp: 'spatial_connectivities', 'spatial_distances'

In [9]:
# Spatial cluster: AgglomerativeClustering + spatial constrain
n_clusters = len(spa_adata.obs.celltype_mapped_refined.unique())
def spatial_cluster(expr, prefix):
    spa_cluster_metrics = {}
    pred_clss = AgglomerativeClustering(n_clusters=n_clusters, 
                                        connectivity=spa_adata.obsp['spatial_connectivities'],
                                        ).fit_predict(expr)
    ars = adjusted_rand_score(spa_adata.obs.celltype_mapped_refined, pred_clss)
    amis = adjusted_mutual_info_score(spa_adata.obs.celltype_mapped_refined, pred_clss)
    homo = homogeneity_score(spa_adata.obs.celltype_mapped_refined, pred_clss)
    nmi = normalized_mutual_info_score(spa_adata.obs.celltype_mapped_refined, pred_clss)
    spa_cluster_metrics[prefix] = {"ARS":ars, "AMIS":amis, 'HOMO': homo, 'NMI':nmi}
    return pd.DataFrame(spa_cluster_metrics)

In [10]:
# column mask for select sc marker genes
candidate_msk = [True  if g in candidate_genes else False for g in np.concatenate([raw_shared_gene, extra_genes])]
np.sum(candidate_msk)

422

In [11]:
"""
Spatial clustering on 
1. seqFISH raw ST data (351 genes)
2. imputed sc celltype marker genes (top 50, 335 genes = 63 extra + 50 overlapped imputed)
3-6. imputed genes selected by prediction confidence scores, top 400, 300, 200, 100

Return averaged clustering scores against manual annotations `celltype_mapped_refined` in spa_adata.obs
"""
df_cls = spatial_cluster(spa_adata.X.toarray(), "SeqFISH_Raw (351) vs Ground.Annotation")
df_cls = pd.concat([df_cls, spatial_cluster(res[0][:, candidate_msk], f"SCImputedMarkers ({len(candidate_genes)}) vs Ground.Annotation")], axis=1)
df_cls = pd.concat([df_cls, spatial_cluster(res[0], f"SCImputedAll ({len(candidate_msk)}) vs Ground.Annotation")], axis=1)
df_cls = pd.concat([df_cls, spatial_cluster(res[0][:, np.argsort(res[1])[:600]], "SCImputedTopConfident (600) vs Ground.Annotation")], axis=1)
df_cls = pd.concat([df_cls, spatial_cluster(res[0][:, np.argsort(res[1])[:500]], "SCImputedTopConfident (500) vs Ground.Annotation")], axis=1)
df_cls = pd.concat([df_cls, spatial_cluster(res[0][:, np.argsort(res[1])[:400]], "SCImputedTopConfident (400) vs Ground.Annotation")], axis=1)
df_cls = pd.concat([df_cls, spatial_cluster(res[0][:, np.argsort(res[1])[:300]], "SCImputedTopConfident (300) vs Ground.Annotation")], axis=1)
df_cls = pd.concat([df_cls, spatial_cluster(res[0][:, np.argsort(res[1])[:200]], "SCImputedTopConfident (200) vs Ground.Annotation")], axis=1)
df_cls = pd.concat([df_cls, spatial_cluster(res[0][:, np.argsort(res[1])[:100]], "SCImputedTopConfident (100) vs Ground.Annotation")], axis=1)
df_cls = pd.concat([df_cls, spatial_cluster(res[0][:, np.argsort(res[1])[:50]], "SCImputedTopConfident (50) vs Ground.Annotation")], axis=1)
df_cls

,SeqFISH_Raw (351) vs Ground.Annotation,SCImputedMarkers (422) vs Ground.Annotation,SCImputedAll (618) vs Ground.Annotation,SCImputedTopConfident (600) vs Ground.Annotation,SCImputedTopConfident (500) vs Ground.Annotation,SCImputedTopConfident (400) vs Ground.Annotation,SCImputedTopConfident (300) vs Ground.Annotation,SCImputedTopConfident (200) vs Ground.Annotation,SCImputedTopConfident (100) vs Ground.Annotation,SCImputedTopConfident (50) vs Ground.Annotation
AMIS,0.339416,0.342975,0.343578,0.343578,0.347258,0.348164,0.346621,0.344788,0.324350,0.305074
ARS,0.183854,0.298666,0.332807,0.332807,0.344139,0.331952,0.328118,0.327552,0.202472,0.192753
HOMO,0.350695,0.331197,0.331486,0.331486,0.339157,0.332940,0.331119,0.327599,0.308563,0.281735
NMI,0.340482,0.344110,0.344714,0.344714,0.348374,0.349302,0.347763,0.345940,0.325529,0.306308


In [12]:
df_cls.mean()

SeqFISH_Raw (351) vs Ground.Annotation              0.303612
SCImputedMarkers (422) vs Ground.Annotation         0.329237
SCImputedAll (618) vs Ground.Annotation             0.338147
SCImputedTopConfident (600) vs Ground.Annotation    0.338147
SCImputedTopConfident (500) vs Ground.Annotation    0.344732
SCImputedTopConfident (400) vs Ground.Annotation    0.340590
SCImputedTopConfident (300) vs Ground.Annotation    0.338405
SCImputedTopConfident (200) vs Ground.Annotation    0.336470
SCImputedTopConfident (100) vs Ground.Annotation    0.290229
SCImputedTopConfident (50) vs Ground.Annotation     0.271468
dtype: float64

In [13]:
df_cls.index.name = 'metric'
df_cls.to_csv("../../output/segfish_cluster_with_extra_genes_top50.csv")